In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request as ur
import random
import time
import json
from selenium.webdriver.common.keys import Keys
import math
BACKSPACE='/ue003'
ENTER='/ue007'

In [38]:
def getID(good_list, bad_list):
    #날짜 설정
#     beg_date = ['1993-01-01', '1996-03-01', '1998-08-01', '2000-08-01',
#                 '2001-07-01', '2002-12-01', '2005-04-01', '2008-01-01',
#                 '2009-02-01', '2011-08-01', '2013-03-01', '2017-09-01']
#     end_date = ['1993-01-31', '1996-03-31', '1998-08-31', '2000-08-31',
#                 '2001-07-31', '2002-12-31', '2005-04-30', '2008-01-31',
#                 '2009-02-28', '2011-08-31', '2013-03-31', '2017-09-30']
    beg_date = ['2017-09-01']
    end_date = ['2017-09-30']
    keywords = ['판매 증가']
    
    #webdriver 객체를 만들어 준다.
    driver = webdriver.Chrome('./chromedriver') 
    
    for k in range(len(beg_date)):
        for key in keywords:
            driver.get('https://www.bigkinds.or.kr')
            # 모든 자원이 로드될때 까지 기다리게 하는 시간을 직접 지정해 준다.
            driver.implicitly_wait(5)
            searchkey = key
            #검색어를 입력한다.
            driver.find_element_by_id('total-search-key').send_keys(searchkey)
            #검색버튼을 누른다.
            driver.find_element_by_class_name('news-search-btn').click()
            driver.implicitly_wait(5) 
            time.sleep(1.5)
            #뉴스 검색버튼을 누른다.
            driver.find_element_by_xpath('//*[@id="headingOne"]/h4/a').click()
            time.sleep(1.5)
            #기간버튼을 누른다.
            driver.find_element_by_id('date-filter-btn').click()
            time.sleep(1)
            #기본값을 지우고 기간을 입력한다.
            driver.find_element_by_id('search-begin-date').click()
            time.sleep(1)
            for i in range(10):
                driver.find_element_by_id('search-begin-date').send_keys(Keys.BACKSPACE)
            driver.find_element_by_id('search-begin-date').send_keys(beg_date[k])
            driver.find_element_by_id('search-begin-date').send_keys(Keys.ENTER)
            time.sleep(1)
            driver.find_element_by_id('search-end-date').click()
            time.sleep(1)
            for i in range(10):
                driver.find_element_by_id('search-end-date').send_keys(Keys.BACKSPACE)
            driver.find_element_by_id('search-end-date').send_keys(end_date[k])
            driver.find_element_by_id('search-begin-date').send_keys(Keys.ENTER)
            time.sleep(1)
            #적용버튼을 누른다.
            driver.find_element_by_xpath('//*[@id="date-confirm-btn"]').click()
            time.sleep(1)
            
            #상세검색 버튼을 누른다.
            driver.find_element_by_xpath('//*[@id="detail-filter-btn"]').click()
            time.sleep(0.5)
            #바이그램을 선택한다.
            driver.find_element_by_xpath('//*[@id="search-filter-type"]/option[2]').click()
            #정확히 일치하는 단어를 입력한다.
            driver.find_element_by_id('exactKeyword1').send_keys(searchkey)
            #검색버튼을 누른다.
            driver.find_element_by_class_name('news-search-btn').click()
            time.sleep(1.5)
            #다음버튼(7개의 페이지 후)을 몇번 누를 것 인지 계산한다.
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            
            #검색 결과 없음
            if(soup.find('span',{'id':'total-news-cnt'}).text == '0'):
                continue
            #알 수 없는 에러 (비슷한 숫자가 변환 안된다는 에러가 뜨면 밑의 숫자를 바꾸어주어야 함)
            temp = 0
            if(',' in soup.find('span',{'id':'total-news-cnt'}).text): 
                temp = soup.find('span',{'id':'total-news-cnt'}).text
                temp = temp[0] + temp[2] + temp[3] + temp[4]
                nextbt = float(temp) / 70
#                 bad = { 'searchkey' : searchkey,
#                         'searchdate' : beg_date[k]
#                 }
#                 bad_list.append(bad)
#                 continue
            else : nextbt = float(soup.find('span',{'id':'total-news-cnt'}).text) / 70
            
#             if(float(soup.find('span',{'id':'total-news-cnt'}).text) % 70 == 0):
#                 nextbt -= 1
            if(float(nextbt) % 70 == 0):
                nextbt -= 1
            print(nextbt)
            for j in range(int(float(nextbt))+1):
                driver.implicitly_wait(7)
                #페이지 링크(1, 2, 3, 4 .. 페이지 번호)를 가지고 온다.
                time.sleep(2)
                pages = driver.find_elements_by_class_name('page-link')
                #페이지 링크(0, 1, 은 처음버튼과 이전 버튼입니다. 접근하지 않습니다.)
                for i in range(3 , len(pages)):
                    driver.implicitly_wait(10)
                    time.sleep(3.5)
                    pages = driver.find_elements_by_class_name('page-link')
                    idlist = []
                    #페이지의 elements를 모두 가지고 온다.
                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')
                    #id가 있는 div로 접근한다.
                    ids = soup.find_all('div', attrs = {'class' :'news-item'})

                    for id in ids:
                        tmp = id.get('data-id')
                        form = {'id' : tmp,
                                'searchkey' : searchkey,
                                'searchdate' : beg_date[k]
                        }
                        good_list.append(form)
                    time.sleep(0.5)
                    pages[i].click()

In [39]:
# good_list = []
# bad_list = []

getID(good_list, bad_list)

1.5571428571428572


In [40]:
len(good_list)

120

In [7]:
bad_list

[]

In [ ]:
good_list

In [41]:
#json 파일로 만들어준다.
json.dump(good_list, open('./Newsid71.json', 'w+'), ensure_ascii=False)
# json.dump(bad_list, open('./Newsbadid10.json', 'w+'), ensure_ascii=False)